In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

geojson с муниципальными образованиями от [gis-lab.info](http://gis-lab.info/qa/moscow-atd.html#.D0.A1.D0.BA.D0.B0.D1.87.D0.B0.D1.82.D1.8C_.D0.B4.D0.B0.D0.BD.D0.BD.D1.8B.D0.B5) или [github.com](https://github.com/blackmad/neighborhoods/blob/master/gn-moscow.geojson)

```bash
$ conda install geopandas
```

Если ошибка во время импорта `fiona`
```bash
$ conda install fiona "libgdal<2.1"
```

In [4]:
shapes = gpd.read_file("./mo.geojson")
shapes = shapes.loc[:,["geometry", "NAME"]]
shapes.head()

,geometry,NAME
0,"(POLYGON ((36.8031 55.44083, 36.80319 55.4416,...",Киевский
1,"POLYGON ((37.42765 55.74821, 37.42849 55.74875...",Филёвский Парк
2,"POLYGON ((36.80357 55.45162, 36.80451 55.46551...",Новофёдоровское
3,"POLYGON ((36.93724 55.24139, 36.93726 55.24161...",Роговское
4,"POLYGON ((37.43956 55.62731, 37.44018 55.63042...","""Мосрентген"""


In [13]:
events = pd.read_table("./data.tsv")
events["geometry"] = events.apply(lambda row: Point(row["lon"], row["lat"]), axis=1)
events = gpd.GeoDataFrame(events, geometry="geometry")
events.head()

,lat,lon,geometry
0,55.80614,37.78192,POINT (37.78192 55.80614)
1,55.82009,37.51120,POINT (37.5112 55.82009)
2,55.59759,39.51329,POINT (39.51329 55.59759)
3,55.80573,37.77863,POINT (37.77863 55.80573)
4,55.77469,37.60546,POINT (37.60546 55.77469)


In [22]:
joined_data = gpd.tools.sjoin(shapes, events, how="inner")
joined_data[~joined_data.NAME.isnull()].head()

In [23]:
joined_data[joined_data.NAME.isnull()].head()

,geometry,NAME,index_right,lat,lon


In [24]:
joined_data[~joined_data.NAME.isnull()].head()

,geometry,NAME,index_right,lat,lon
0,"(POLYGON ((36.8031 55.44083, 36.80319 55.4416,...",Киевский,100,55.42901,36.86551
0,"(POLYGON ((36.8031 55.44083, 36.80319 55.4416,...",Киевский,33203,55.42642,36.86566
0,"(POLYGON ((36.8031 55.44083, 36.80319 55.4416,...",Киевский,29688,55.42756,36.86603
0,"(POLYGON ((36.8031 55.44083, 36.80319 55.4416,...",Киевский,9928,55.42606,36.86663
0,"(POLYGON ((36.8031 55.44083, 36.80319 55.4416,...",Киевский,27602,55.43028,36.86735


In [37]:
import fiona; fiona.supported_drivers

{'ARCGEN': 'r',
 'AeronavFAA': 'r',
 'BNA': 'raw',
 'DGN': 'raw',
 'DXF': 'raw',
 'ESRI Shapefile': 'raw',
 'FileGDB': 'raw',
 'GMT': 'raw',
 'GPKG': 'rw',
 'GPSTrackMaker': 'raw',
 'GPX': 'raw',
 'GeoJSON': 'rw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'OpenFileGDB': 'r',
 'PCIDSK': 'r',
 'PDS': 'r',
 'SEGY': 'r',
 'SUA': 'r'}

In [43]:
joined_data.to_file("joined_events", encoding="utf-8")

In [44]:
# !open ./joined_events/joined_events.shp -a tableau

![opt](options.png)
![viz](viz.png)